In [25]:
cd("C:/Users/neugg/OneDrive/Documents/Frankly AI/GitHub/medium-articles/julia-vs-python/")

In [26]:
#=
# Import Packages
using Pkg  # Package to install new packages

# Install packages 
Pkg.add("DataFrames")
Pkg.add("CSV")
Pkg.add("Lathe")
Pkg.add("GLM")
=#

# Load the installed packages
using DataFrames
using CSV
using Lathe
using Lathe.preprocess: TrainTestSplit
using GLM

In [27]:
insurance_df = DataFrame(CSV.File("data/insurance.csv"))
describe(insurance_df)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,age,39.207,18,39.0,64,0,Int64
2,sex,,female,,male,0,String
3,bmi,30.6634,15.96,30.4,53.13,0,Float64
4,children,1.09492,0,1.0,5,0,Int64
5,smoker,,no,,yes,0,String
6,region,,northeast,,southwest,0,String
7,charges,13270.4,1121.87,9382.03,63770.4,0,Float64


In [28]:
println(first(insurance_df,5))

5×7 DataFrame
 Row │ age    sex     bmi      children  smoker  region     charges  
     │ Int64  String  Float64  Int64     String  String     Float64  
─────┼───────────────────────────────────────────────────────────────
   1 │    19  female   27.9           0  yes     southwest  16884.9
   2 │    18  male     33.77          1  no      southeast   1725.55
   3 │    28  male     33.0           3  no      southeast   4449.46
   4 │    33  male     22.705         0  no      northwest  21984.5
   5 │    32  male     28.88          0  no      northwest   3866.86


In [29]:
println("The shape of the dataset is:", size(insurance_df))
describe(insurance_df)

The shape of the dataset is:(1338, 7)


,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,age,39.207,18,39.0,64,0,Int64
2,sex,,female,,male,0,String
3,bmi,30.6634,15.96,30.4,53.13,0,Float64
4,children,1.09492,0,1.0,5,0,Int64
5,smoker,,no,,yes,0,String
6,region,,northeast,,southwest,0,String
7,charges,13270.4,1121.87,9382.03,63770.4,0,Float64


In [30]:
# One hot encoding
lathe_one_hot = Lathe.preprocess.OneHotEncoder()
scaled_feature = lathe_one_hot.predict(insurance_df, :region)
select!(insurance_df, Not([:region]))
println("\nThe encoded dataframe sample:\n", first(insurance_df,5))


The encoded dataframe sample:
5×10 DataFrame
 Row │ age    sex     bmi      children  smoker  charges   southwest  southeast  northwest  northeast 
     │ Int64  String  Float64  Int64     String  Float64   Bool       Bool       Bool       Bool      
─────┼────────────────────────────────────────────────────────────────────────────────────────────────
   1 │    19  female   27.9           0  yes     16884.9        true      false      false      false
   2 │    18  male     33.77          1  no       1725.55      false       true      false      false
   3 │    28  male     33.0           3  no       4449.46      false       true      false      false
   4 │    33  male     22.705         0  no      21984.5       false      false       true      false
   5 │    32  male     28.88          0  no       3866.86      false      false       true      false


In [31]:
train, test = TrainTestSplit(insurance_df,.75)

(1016×10 DataFrame
  Row │ age    sex     bmi      children  smoker  charges   southwest  southea ⋯
      │ Int64  String  Float64  Int64     String  Float64   Bool       Bool    ⋯
──────┼─────────────────────────────────────────────────────────────────────────
    1 │    19  female   27.9           0  yes     16884.9        true      fal ⋯
    2 │    18  male     33.77          1  no       1725.55      false       tr
    3 │    28  male     33.0           3  no       4449.46      false       tr
    4 │    33  male     22.705         0  no      21984.5       false      fal
    5 │    32  male     28.88          0  no       3866.86      false      fal ⋯
    6 │    31  female   25.74          0  no       3756.62      false       tr
    7 │    37  female   27.74          3  no       7281.51      false      fal
    8 │    37  male     29.83          2  no       6406.41      false      fal
  ⋮   │   ⋮      ⋮        ⋮        ⋮        ⋮        ⋮          ⋮          ⋮   ⋱
 1010 │    57  female

In [32]:
fm = @formula(charges ~ age + bmi + children + sex + smoker + southwest + southeast + northwest + northeast)
linear_regressor = lm(fm, train)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

charges ~ 1 + age + bmi + children + sex + smoker + southwest + southeast + northwest + northeast

Coefficients:
─────────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error       t  Pr(>|t|)   Lower 95%   Upper 95%
─────────────────────────────────────────────────────────────────────────────
(Intercept)       0.0      NaN       NaN       NaN        NaN         NaN
age             259.866     13.2976   19.54    <1e-71     233.772     285.96
bmi             333.345     31.6253   10.54    <1e-23     271.286     395.404
children        426.158    156.145     2.73    0.0065     119.752     732.564
sex: male      -398.983    373.616    -1.07    0.2858   -1132.14      334.173
smoker: yes   23425.3      465.277    50.35    <1e-99   22512.3     24338.3
southwest    -

In [33]:
println("\nR2 for the linear regressor: ", r2(linear_regressor))


R2 for the linear regressor: 0.7514530856688412
